# Работа со строковыми значениями

__Автор задач: Блохин Н.В. (NVBlokhin@fa.ru)__

Материалы:
* Макрушин С.В. Лекция "Работа со строковыми значениям"
* https://pyformat.info/
* https://docs.python.org/3/library/re.html
    * https://docs.python.org/3/library/re.html#flags
    * https://docs.python.org/3/library/re.html#functions
* https://pythonru.com/primery/primery-primeneniya-regulyarnyh-vyrazheniy-v-python
* https://kanoki.org/2019/11/12/how-to-use-regex-in-pandas/
* https://realpython.com/nltk-nlp-python/

## Задачи для совместного разбора

1. Вывести на экран данные из словаря `obj` построчно в виде `k = v`, задав формат таким образом, чтобы знак равенства оказался на одной и той же позиции во всех строках. Строковые литералы обернуть в кавычки.

In [1]:
obj = {
    "home_page": "https://github.com/pypa/sampleproject",
    "keywords": "sample setuptools development",
    "license": "MIT",
}

2. Написать регулярное выражение,которое позволит найти номера групп студентов.

In [4]:
obj = pd.Series(["Евгения гр.ПМ19-1", "Илья пм 20-4", "Анна 20-3"])
obj

0    Евгения гр.ПМ19-1
1         Илья пм 20-4
2            Анна 20-3
dtype: object

3. Разбейте текст формулировки задачи 2 на слова.

## Лабораторная работа 6

In [61]:
import numpy as np
import pandas as pd
import csv
import requests
from bs4 import BeautifulSoup

### Форматирование строк

1\. Загрузите данные из файла `recipes_sample.csv` (__ЛР2__) в виде `pd.DataFrame` `recipes` При помощи форматирования строк выведите информацию об id рецепта и времени выполнения 5 случайных рецептов в виде таблицы следующего вида:

    
    |      id      |  minutes  |
    |--------------------------|
    |    61178     |    65     |
    |    202352    |    80     |
    |    364322    |    150    |
    |    26177     |    20     |
    |    224785    |    35     |
    
Обратите внимание, что ширина столбцов заранее неизвестна и должна рассчитываться динамически, в зависимости от тех данных, которые были выбраны. 

In [45]:
recipes = pd.read_csv('recipes_sample.csv', delimiter=',')
samplee=pd.DataFrame(recipes.sample(n=5))[['id', 'minutes']]
k=0
for i in samplee['id']:
    if len(str(i)) > k:
        k=len(str(i))

a= f'|{"id":^{k+8}}|{"minutes":^{k+8}}|'
b='|'+ '-'*(len(a)-2) +'|'
print(a)
print(b)
for i, j in zip(samplee['id'], samplee['minutes']):
        print(f'|{ i :^{k+8}}|{ j :^{k+8}}|')



|      id      |   minutes    |
|-----------------------------|
|    219626    |      15      |
|    150334    |      25      |
|    334626    |      25      |
|    311708    |      10      |
|    153967    |      5       |


2\. Напишите функцию `show_info`, которая по данным о рецепте создает строку (в смысле объекта python) с описанием следующего вида:

```
"Название Из Нескольких Слов"

1. Шаг 1
2. Шаг 2
----------
Автор: contributor_id
Среднее время приготовления: minutes минут
```

    
Данные для создания строки получите из файлов `recipes_sample.csv` (__ЛР2__) и `steps_sample.xml` (__ЛР3__). 
Вызовите данную функцию для рецепта с id `170895` и выведите (через `print`) полученную строку на экран.

In [57]:
def show_info(recipe_id):
    # Считываем данные о рецепте из файла recipes_sample.csv
    with open('recipes_sample.csv', encoding='utf-8') as f:
        reader = csv.reader(f, delimiter=',')
        headers = next(reader) # пропускаем заголовок
        for row in reader:
            if row[0] == recipe_id:
                recipe_name = row[1]
                contributor_id = row[2]
                minutes = row[3]
                break
    
    # Считываем данные о шагах приготовления из файла steps_sample.xml
    tree = ET.parse('steps_sample.xml')
    root = tree.getroot()
    for recipe in root.findall('recipe'):
        if recipe.attrib['id'] == recipe_id:
            steps = [step.text for step in recipe.findall('step')]
            break
    
    # Формируем строку с описанием рецепта
    recipe_name = recipe_name.title()
    steps_str = '\n'.join([f'{i}. {step}' for i, step in enumerate(steps, start=1)])
    info_str = f'{recipe_name}\n\n{steps_str}\n{"-"*10}\nАвтор: {contributor_id}\nСреднее время приготовления: {minutes} минут'
    return info_str

recipe_id = '170895'
recipe_info = show_info(recipe_id)
print(recipe_info)

KeyError: 'id'

In [58]:
assert (
    show_info(
        name="george s at the cove black bean soup",
        steps=[
            "clean the leeks and discard the dark green portions",
            "cut the leeks lengthwise then into one-inch pieces",
            "melt the butter in a medium skillet , med",
        ],
        minutes=90,
        author_id=35193,
    )
    == '"George S At The Cove Black Bean Soup"\n\n1. Clean the leeks and discard the dark green portions\n2. Cut the leeks lengthwise then into one-inch pieces\n3. Melt the butter in a medium skillet , med\n----------\nАвтор: 35193\nСреднее время приготовления: 90 минут\n'
)

TypeError: show_info() got an unexpected keyword argument 'name'

## Работа с регулярными выражениями

In [64]:
import re

3\. Напишите регулярное выражение, которое ищет следующий паттерн в строке: число (1 цифра или более), затем пробел, затем слова: hour или hours или minute или minutes. Произведите поиск по данному регулярному выражению в каждом шаге рецепта с id 25082. Выведите на экран все непустые результаты, найденные по данному шаблону.

\d+ соответствует одной или нескольким цифрам

\s+ соответствует одному или нескольким пробелам

(hours?|minutes?) соответствует словам "hour", "hours", "minute" или "minutes", где символ ? указывает на то, что символ s в слове hours может быть как присутствующим, так и отсутствующим.

In [65]:
pattern = r'\d+\s+(hours?|minutes?)'
with open('steps_sample.xml') as f:
    ab_1 = BeautifulSoup(f, 'xml')

ids = ab_1.find_all('id')
steps = ab_1.find_all('steps')
dct_res = {}
for i in range(len(ids)):
    dct_res[ids[i].getText(strip=True)]=(steps[i].getText(',',strip=True)).split(',')
    
m= dct_res.get('25082')
for i in m:
    match = re.search(pattern, i)
    if match:
        print(match.group())


20 minutes
10 minutes
2 hours
10 minutes
20 minutes
30 minutes


4\. Напишите регулярное выражение, которое ищет шаблон вида "this..., but" _в начале строки_ . Между словом "this" и частью ", but" может находиться произвольное число букв, цифр, знаков подчеркивания и пробелов. Никаких других символов вместо многоточия быть не может. Пробел между запятой и словом "but" может присутствовать или отсутствовать.

Используя строковые методы `pd.Series`, выясните, для каких рецептов данный шаблон содержится в тексте описания. Выведите на экран количество таких рецептов и 3 примера подходящих описаний (текст описания должен быть виден на экране полностью).

In [70]:
pattern = r'^this[\w\s]*\.{3}\s*,?\s*but'
recipes = pd.read_csv('recipes_sample.csv', delimiter=',')
matches = recipes['description'].str.contains(pattern, regex=True)
num_matches = sum(matches)
print(f'Количество рецептов, содержащих шаблон: {num_matches}')
print('Примеры описаний, соответствующих шаблону:')
print(recipes[matches]['description'].head(3))

Количество рецептов, содержащих шаблон: nan
Примеры описаний, соответствующих шаблону:


ValueError: Cannot mask with non-boolean array containing NA / NaN values

5\. В текстах шагов рецептов обыкновенные дроби имеют вид "a / b". Используя регулярные выражения, уберите в тексте шагов рецепта с id 72367 пробелы до и после символа дроби. Выведите на экран шаги этого рецепта после их изменения.

In [76]:
m= " ".join(dct_res.get('72367'))
steps = re.sub(r'\s*/\s*', '/', m)
print(steps)

mix butter   flour   1/3 c sugar and 1-1/4 t vanilla press into greased 9" springform pan mix cream cheese   1/4 c sugar   eggs and 1/2 t vanilla beating until fluffy pour over dough combine apples   1/3 c sugar and cinnamon arrange on top of cream cheese mixture and sprinkle with almonds bake at 350 for 45-55 minutes   or until tester comes out clean


### Сегментация текста

In [86]:
import nltk
from nltk.tokenize import word_tokenize

6\. Разбейте тексты шагов рецептов на слова при помощи пакета `nltk`. Посчитайте и выведите на экран кол-во уникальных слов среди всех рецептов. Словом называется любая последовательность алфавитных символов (для проверки можно воспользоваться `str.isalpha`). При подсчете количества уникальных слов не учитывайте регистр.

In [90]:
reci = []
for i in list(dct_res.values()):
    a=" ".join(i)
    reci.append(a)

recipe = " ".join(reci) 
print(recipe)
#unique_words = set()

#for word in nltk.word_tokenize(recipe.lower()):
    #if word.isalpha():
        #unique_words.add(word)

#print(len(unique_words))

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



7\. Разбейте описания рецептов из `recipes` на предложения при помощи пакета `nltk`. Найдите 5 самых длинных описаний (по количеству _предложений_) рецептов в датасете и выведите строки фрейма, соответствующие этим рецептами, в порядке убывания длины.

8\. Напишите функцию, которая для заданного предложения выводит информацию о частях речи слов, входящих в предложение, в следующем виде:
```
PRP   VBD   DT      NNS     CC   VBD      NNS        RB   
 I  omitted the raspberries and added strawberries instead
``` 
Для определения части речи слова можно воспользоваться `nltk.pos_tag`.

Проверьте работоспособность функции на названии рецепта с id 241106.

Обратите внимание, что часть речи должна находиться ровно посередине над соотвествующим словом, а между самими словами должен быть ровно один пробел.
